In [ ]:
!apt-get install cuda-9.2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libcuda-9.2-1' for regex 'cuda-9.2'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


BELLMAN-FORD ALGORITHM
SHIVAM CS22M112

In [9]:
%%writefile bellmonford.cu
#include <iostream>
#include <cuda.h>
#include <stdio.h>
#include <bits/stdc++.h>
using namespace std;

__global__ void bellmonford(int dv[], int de[], int dw[], int di[], int dd[])
{
    int id = threadIdx.x;
    if (dd[id] != INT_MAX)
    {
        for (int i = di[id]; i < di[id + 1]; i++)
        {
            int v = de[i];
            if (dd[v] > dd[id] + dw[i])
            {
                dd[v] = dd[id] + dw[i];
            }
        }
    }
}

int main()
{
    int n, m, s;
    cout << "enter the no of vertex";
    cin >> n;
    cout << "enter the no of edge";
    cin >> m;
    cout << "enter the source";
    cin >> s;

    int *v = new int[n];
    int *e = new int[m];
    int *index = new int[n+1];
    int *w = new int[m];
    int *d = new int[n];

    for (int i = 0; i < n; i++)
    {
        v[i] = i;
    }
    cout << "index"
         << " ";
    for (int i = 0; i < n; i++)
    {
        int q;
        cin >> q;
        index[i] = q;
    }
    index[n]=m;
    cout << "edge"
         << " ";
    for (int i = 0; i < m; i++)
    {
        int q;
        cin >> q;
        e[i] = q;
    }
    cout << "weight"
         << " ";
    for (int i = 0; i < m; i++)
    {
        int q;
        cin >> q;
        w[i] = q;
    }
    cout << "distance"
         << " ";
    d[s] = 0;
    for (int i = 0; i < n; i++)
    {
        if (i != s)
        {
            d[i] = INT_MAX;
        }
    }

    int *dv;
    int *de;
    int *dindex;
    int *dw;
    int *dd;
    cudaMalloc(&dv, n * sizeof(int));
    cudaMalloc(&de, m * sizeof(int));
    cudaMalloc(&dw, m * sizeof(int));
    cudaMalloc(&dd, n * sizeof(int));
    cudaMalloc(&dindex, (n+1) * sizeof(int));

    cudaMemcpy(dv, v, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(de, e, m * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dw, w, m * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dindex, index, (n+1) * sizeof(int), cudaMemcpyHostToDevice);

    for (int i = 0; i < (n); i++)
    {
        cudaMemcpy(dd, d, n * sizeof(int), cudaMemcpyHostToDevice);
        bellmonford<<<1, n>>>(dv, de, dw, dindex, dd);
        cudaMemcpy(d, dd, n * sizeof(int), cudaMemcpyDeviceToHost);
    }
    for (int i = 0; i < n; i++)
    {
        if(d[i]==INT_MAX)
          cout<<"INF"<<" ";
        else
          cout << d[i] << " ";
    }

    cudaFree(dv);
    cudaFree(de);
    cudaFree(dindex);
    cudaFree(dd);
    cudaFree(dw);
    return (0);
}

Overwriting bellmonford.cu


In [10]:
!nvcc bellmonford.cu
!./a.out

enter the no of vertex6
enter the no of edge11
enter the source5
index 0 2 5 7 8 9
edge 1 2 2 3 4 3 4 1 3 3 4
weight 9 4 10 2 3 2 11 2 2 10 15
distance INF 12 22 10 15 0 